# Question 1

Are contributors of comments (commit comments, issue comments, pull request comments) more likely to contribute to packages they depend on (or to packages that depend on them) than to other packages?

*** I think to answer this question we need all comments of users who have contributed on a package... to this end, we need to scrap github to find all comments of a user